# 2.7.2: Glucose (In class)

Use this notebook to help you complete the PAL.   The first cells include the code from the Glucose notebooks; you don't need to change any of those cells.  When you want to test different parameters, use the last two cells.

In [ ]:
# Import the glucose and insulin data
import pandas as pd
import numpy as np

filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/glucose_insulin.xlsx'
data = pd.read_excel(filename, index_col=0)
data.columns = ['glucose', 'insulin']

In [ ]:
# Define the change function and the run_simulation function
def change_func(t, state, system):
    G, X = state
    k1, k2, k3, dt = system['k1'],system['k2'], system['k3'], system['dt']
    I, Ib, Gb = system['I'], system['Ib'], system['Gb']

    dGdt = -k1 * (G - Gb) - X*G
    dXdt = k3 * (I[t] - Ib) - k2 * X

    state.G += dGdt * dt
    state.X += dXdt * dt

    return state

def run_simulation(system, state, change_func):
    t_array = np.arange(system['t_0'], system['t_end']+1, system['dt'])
    n = len(t_array)

    results = pd.DataFrame(index=t_array, columns=state.index,
                        dtype=np.float64)
    results.iloc[0] = state

    for i in range(n-1):
        t = t_array[i]
        results.iloc[i+1] = change_func(t, state, system)

    return results

In [ ]:
# Define parameters (from the notebook)
G0 = 270
X0 = 0
k1 = 0.02
k2 = 0.02
k3 = 1.5e-05

# Define simulation parameters based on data
t_0 = data.index[0]
t_end = data.index[-1]
Gb = data.glucose[t_0]
Ib = data.insulin[t_0]
dt = (t_end-t_0)/(data.glucose.size -1)

# Define the state and system
state = pd.Series(dict(G=G0, X=X0),dtype=np.float64)
system = dict(init=state, k1=k1,k2=k2,
                  k3=k3,dt=dt, Gb=Gb, Ib=Ib, I=data.insulin,
                  t_0=t_0, t_end=t_end)

# Run the simulation (remember that 'results' is a DataFrame)
results = run_simulation(system, state,change_func)

In [ ]:
# Plot the results
ax_second = results.plot(y='G', color='C0',
             label='glucose (mg/dL)',ylabel='(mg/dL)', legend=True)

# Plots both the primary data 'X' and the secondary axis 'ax_second'
ax = results.plot(y='X', color='C1', secondary_y=True, ax=ax_second,
             label='insulin in tissue fluid', xlabel='Time (min)', style='--',
               legend=True);

# Set limit of right axis
# This uses the MatPlotLib library: the plot() we usually use is based on
# this code, but using MatPlotLib gives the flexibility to include
# a second y-axis
fig = ax.get_figure()
ax = fig.get_axes()
ax[1].set_ylim(0,0.016)

# Plot the known data
data.insulin.plot(label="insulin in blood", color='C2',legend=True);

## Playing with the Model

In the two cells below, you can change the parameters in the first cell, and run and plot the simulation using the second cell.  You are welcome to click "Show code" to see the code in the second cell, but it is just a duplicate of what is above.

In [ ]:
G0 = 270; k1 = 0.02; k2 = 0.02; k3 = 1.5e-05;

In [ ]:
# @title
state = pd.Series(dict(G=G0, X=X0),dtype=np.float64)
system = dict(init=state, k1=k1,k2=k2,
                  k3=k3,dt=dt, Gb=Gb, Ib=Ib, I=data.insulin,
                  t_0=t_0, t_end=t_end)
results = run_simulation(system, state,change_func)

# Creates an 'axis' object that gets carried into the next plot() function
ax_second = results.plot(y='G', color='C0',
             label='glucose (mg/dL)',ylabel='(mg/dL)', legend=True)

# Plots both the primary data 'X' and the secondary axis 'ax_second'
ax = results.plot(y='X', color='C1', secondary_y=True, ax=ax_second,
             label='insulin in tissue fluid', xlabel='Time (min)', style='--',
               legend=True);
# Set limit of right axis
fig = ax.get_figure()
ax = fig.get_axes()
ax[1].set_ylim(0,0.016)

data.insulin.plot(label="insulin in blood", color='C2',legend=True);